# Handy
## Select ROI

ROI (region of interest) is a term of computer vision that defines the portion of an image that will be processed.

This notebook allows you to select:
* ROI - region of interest - defines the area that the model will be fitted with.
* T-ROI - "trigger region of interest" - defines the area that causes the FPS to switch to the fast mode if something moves in it. Used to detect gestures before ROI to then make it immediate there.
* G-ROI - "gesture region of interest" - detected the area that person has to stand in to be considered as target.

Assert: ROI > T-ROI > G-ROI

In [11]:
import sys
sys.path.append('..')
from config import CONFIG

In [12]:
import json
import logging
from os import path
import cv2

rectangle = []
drawing = False

HANDY_WINDOW = "Handy"

def draw_rectangle(event, x, y, flags, param):
    global rectangle
    global drawing
    if event == cv2.EVENT_LBUTTONDOWN:
        # Left mouse button down - store the starting point
        drawing = True
        rectangle = [x, y, x, y]
    elif event == cv2.EVENT_MOUSEMOVE:
        # Move has moved - update the rectangle
        if drawing:
            rectangle[2], rectangle[3] = x, y
    elif event == cv2.EVENT_LBUTTONUP:
        # Left mouse button up - end drawing
        drawing = False


def select_roi(title: str):
    # Init the video feed
    cap = cv2.VideoCapture(CONFIG.stream_url, cv2.CAP_FFMPEG)

    if not cap.isOpened():
        logging.error("VideoCapture not opened")
        exit(-1)

    cv2.namedWindow(HANDY_WINDOW)
    cv2.setMouseCallback(HANDY_WINDOW, draw_rectangle)

    while True:
        ret, frame = cap.read()
        if not ret or frame is None:
            print("Corrupted frame, retrying...")
            continue

        # Resize frame
        frame = cv2.resize(frame, (CONFIG.resize_width, CONFIG.resize_height))

        cv2.putText(frame, title, (0, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 3)
        cv2.putText(frame, "When you're ready, press 'q'", (0, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 3)
        # If there's a full rectangle
        if len(rectangle) == 4:
            cv2.rectangle(frame, (rectangle[0], rectangle[1]), (rectangle[2], rectangle[3]), (0, 255, 0), 2)
        
        cv2.imshow(HANDY_WINDOW, frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

    print(f"Selected: {rectangle}")
    cap.release()
    cv2.destroyAllWindows()

    return rectangle


In [13]:
# Select ROI
ROI = select_roi("Please select ROI (region of interest)")

# Clear global variables
rectangle = []
drawing = False

if ROI:
    print(f"Selected: {ROI}")
    with open(path.join("..", "ROI.json"), "w") as f:
        ROI = {
            "x1": ROI[0],
            "y1": ROI[1],
            "x2": ROI[2],
            "y2": ROI[3],
        }
        json.dump(ROI, f)
else:
    print("Didn't select the ROI!")

Selected: [238, 99, 575, 531]
Selected: [238, 99, 575, 531]


In [14]:
# Select trigger ROI
TROI = select_roi("Please select T-ROI (trigger region of interest)")

# Clear global variables
rectangle = []
drawing = False

if TROI:
    print(f"Selected: {TROI}")
    with open(path.join("..", "TROI.json"), "w") as f:
        TROI = {
            "x1": TROI[0],
            "y1": TROI[1],
            "x2": TROI[2],
            "y2": TROI[3],
        }
        json.dump(TROI, f)
else:
    print("Didn't select the T-ROI!")

Selected: [282, 314, 573, 530]
Selected: [282, 314, 573, 530]


In [15]:
# Select gesture ROI
GROI = select_roi("Please select G-ROI (gesture region of interest)")

# Clear global variables
rectangle = []
drawing = False

if GROI:
    print(f"Selected: {GROI}")
    with open(path.join("..", "GROI.json"), "w") as f:
        GROI = {
            "x1": GROI[0],
            "y1": GROI[1],
            "x2": GROI[2],
            "y2": GROI[3],
        }
        json.dump(GROI, f)
else:
    print("Didn't select the G-ROI!")

Selected: [360, 400, 472, 478]
Selected: [360, 400, 472, 478]
